<a href="https://colab.research.google.com/github/AndjelijaMladenovic00/Deep_Learning/blob/main/GAN_inpainting_with_augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Image inpainting using GAN

In [45]:
# !unzip dataset.zip
import tensorflow as tf
from keras import layers
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv
import os
from tqdm import tqdm
from keras.preprocessing.image import img_to_array
import time

In [46]:
IMAGE_SIZE = 256
BATCH_SIZE = 64

In [47]:
# Definisanje klase za učitavanje slika.
class ImageDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, rgb_path, grayscale_path, batch_size, image_size, image_names, augment=False):
        self.rgb_path = rgb_path
        self.grayscale_path = grayscale_path
        self.batch_size = batch_size
        self.image_size = image_size
        self.image_names = image_names
        self.indexes = np.arange(len(self.image_names))
        self.augment = augment

    def __len__(self):
        return len(self.image_names) // self.batch_size

    def __getitem__(self, index):
        batch_indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        batch_rgb_images = []
        batch_grayscale_images = []

        for i in batch_indexes:
            rgb_image = cv.imread(self.rgb_path + self.image_names[i])
            rgb_image = cv.cvtColor(rgb_image, cv.COLOR_BGR2RGB)
            rgb_image = cv.resize(rgb_image, (self.image_size, self.image_size))
            rgb_image = rgb_image.astype("float32") / 255.0

            grayscale_image = cv.imread(self.grayscale_path + self.image_names[i], cv.IMREAD_GRAYSCALE)
            grayscale_image = cv.resize(grayscale_image, (self.image_size, self.image_size))
            grayscale_image = grayscale_image.astype("float32") / 255.0
            grayscale_image = np.expand_dims(grayscale_image, axis=-1)

            if self.augment:
                rgb_image, grayscale_image = self.augment_data(rgb_image, grayscale_image)

            batch_rgb_images.append(rgb_image)
            batch_grayscale_images.append(grayscale_image)

        return np.array(batch_grayscale_images), np.array(batch_rgb_images)

    def on_epoch_end(self):
        np.random.shuffle(self.indexes)

    def augment_data(self, rgb_image, grayscale_image):
        seed = np.random.randint(1e6)
        tf.random.set_seed(seed)

        # Flip.
        rgb_image = tf.image.random_flip_left_right(rgb_image)
        rgb_image = tf.image.random_flip_up_down(rgb_image)
        grayscale_image = tf.image.random_flip_left_right(grayscale_image)
        grayscale_image = tf.image.random_flip_up_down(grayscale_image)

        # Rotacija.
        angles = [0, 90, 180, 270]
        angle = np.random.choice(angles)
        rgb_image = tf.image.rot90(rgb_image, k=angle // 90)
        grayscale_image = tf.image.rot90(grayscale_image, k=angle // 90)

        # Gaussian Noise na ulaznim slikama.
        grayscale_image += tf.random.normal(shape=tf.shape(grayscale_image), mean=0.0, stddev=0.05, dtype=tf.float32)

        return rgb_image, grayscale_image

In [48]:
# Učitavanje putanja slika.
rgb_path = "dataset/color/"
grayscale_path = "dataset/gray/"
image_names = os.listdir(rgb_path)
image_names = sorted(image_names, key=lambda x: int(x.split('.')[0]))

In [49]:
# Podela podataka na trening i test skup podataka i pravljenje generatora podataka.
TRAINING_SIZE = int(len(image_names) * 0.8)
train_image_names = image_names[:TRAINING_SIZE]
test_image_names = image_names[TRAINING_SIZE:]

train_generator = ImageDataGenerator(rgb_path, grayscale_path, BATCH_SIZE, IMAGE_SIZE, train_image_names, augment=True)
test_generator = ImageDataGenerator(rgb_path, grayscale_path, 8, IMAGE_SIZE, test_image_names, augment=False)

In [50]:
# Definisanje funkcija za dodavanje slojeva mreži.
def add_downsampling_layer(filters, size, normalize=True):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Conv2D(filters, size, strides=2, padding="same", kernel_initializer="he_normal", use_bias=False))
    if normalize:
        model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU())
    return model

def add_upsampling_layer(filters, size, dropout=False):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Conv2DTranspose(filters, size, strides=2, padding="same", kernel_initializer="he_normal", use_bias=False))
    model.add(tf.keras.layers.BatchNormalization())
    if dropout:
        model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.LeakyReLU())
    return model

In [51]:
# Definisanje generatora.
def make_generator():
    inputs = tf.keras.layers.Input(shape=[256, 256, 1])
    downsampling_layers = [
        add_downsampling_layer(64, 4, normalize=False),
        add_downsampling_layer(128, 4),
        add_downsampling_layer(256, 4),
        add_downsampling_layer(512, 4),
        add_downsampling_layer(512, 4),
        add_downsampling_layer(512, 4),
        add_downsampling_layer(512, 4),
        add_downsampling_layer(512, 4),
    ]
    upsampling_layers = [
        add_upsampling_layer(512, 4, dropout=True),
        add_upsampling_layer(512, 4, dropout=True),
        add_upsampling_layer(512, 4, dropout=True),
        add_upsampling_layer(512, 4),
        add_upsampling_layer(256, 4),
        add_upsampling_layer(128, 4),
        add_upsampling_layer(64, 4),
    ]
    last_layer = tf.keras.layers.Conv2DTranspose(3, 4, strides=2, padding="same", kernel_initializer=tf.random_normal_initializer(0., 0.02), activation="tanh")

    x = inputs
    downsampled = []
    for layer in downsampling_layers:
        x = layer(x)
        downsampled.append(x)
    downsampled = downsampled[:-1][::-1]

    for i in range(len(upsampling_layers)):
        x = upsampling_layers[i](x)
        x = tf.keras.layers.Concatenate()([x, downsampled[i]])

    x = last_layer(x)
    return tf.keras.Model(inputs=inputs, outputs=x)

In [52]:
# Definisanje diskriminatora.
def make_discriminator():
    image = tf.keras.layers.Input(shape=[IMAGE_SIZE, IMAGE_SIZE, 1], name="image")
    target = tf.keras.layers.Input(shape=[IMAGE_SIZE, IMAGE_SIZE, 3], name="target")
    inputs = tf.keras.layers.concatenate([image, target])

    downsampled1 = add_downsampling_layer(64, 4, False)(inputs)
    downsampled2 = add_downsampling_layer(128, 4)(downsampled1)
    downsampled3 = add_downsampling_layer(256, 4)(downsampled2)

    padding = tf.keras.layers.ZeroPadding2D()(downsampled3)
    convolution = tf.keras.layers.Conv2D(512, 4, strides=1, kernel_initializer=tf.random_normal_initializer(0., 0.02), use_bias=False)(padding)
    normalization = tf.keras.layers.BatchNormalization()(convolution)
    activation = tf.keras.layers.LeakyReLU()(normalization)
    padding2 = tf.keras.layers.ZeroPadding2D()(activation)
    final = tf.keras.layers.Conv2D(1, 4, strides=1, kernel_initializer=tf.random_normal_initializer(0., 0.02))(padding2)

    return tf.keras.Model(inputs=[image, target], outputs=final)

In [53]:
# Definisanje loss funkcija.
losses = tf.keras.losses.BinaryCrossentropy(from_logits=True)
LAMBDA = 100

def generator_loss(disc_output, gen_output, target):
    gan_loss = losses(tf.ones_like(disc_output), disc_output)
    l1_loss = tf.reduce_mean(tf.abs(target - gen_output))
    gen_loss = gan_loss + LAMBDA * l1_loss
    return gen_loss, gan_loss, l1_loss

def discriminator_loss(real, generated):
    real_loss = losses(tf.ones_like(real), real)
    generated_loss = losses(tf.zeros_like(generated), generated)
    disc_loss = real_loss + generated_loss
    return disc_loss

In [54]:
# Inicijalizacija modela.
generator = make_generator()
discriminator = make_discriminator()

In [55]:
# Inicijalizacija optimizatora.
gen_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
disc_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)

In [56]:
# Definisanje jednog koraka treninga.
@tf.function
def training_step(image, target):
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        gen_output = generator(image, training=True)
        disc_real_output = discriminator([image, target], training=True)
        disc_generated_output = discriminator([image, gen_output], training=True)

        gen_loss, gan_loss, l1_loss = generator_loss(disc_generated_output, gen_output, target)
        disc_loss = discriminator_loss(disc_real_output, disc_generated_output)

        gen_gradients = gen_tape.gradient(gen_loss, generator.trainable_variables)
        disc_gradients = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

        gen_optimizer.apply_gradients(zip(gen_gradients, generator.trainable_variables))
        disc_optimizer.apply_gradients(zip(disc_gradients, discriminator.trainable_variables))

In [57]:
# Definisanje fit metode.
def fit(train_generator, num_epochs):
    for epoch in range(num_epochs):
        start_time = time.time()
        print("Epoch: ", epoch + 1)
        for batch in train_generator:
            grayscale_images, rgb_images = batch
            training_step(grayscale_images, rgb_images)
        print(f"Time taken for epoch {epoch + 1} is {time.time() - start_time} sec\n")


In [58]:
# Definisanje metode za testiranje i evaluaciju.
def test_and_evaluate(test_generator):
    average_loss = 0
    num_batches = len(test_generator)
    for batch in tqdm(test_generator):
        grayscale_images, rgb_images = batch
        predictions = generator(grayscale_images, training=False)
        _, _, l1_loss = generator_loss(discriminator([grayscale_images, predictions], training=False), predictions, rgb_images)
        average_loss += l1_loss

    average_loss /= num_batches
    return average_loss

In [60]:
# Treniranje modela.
fit(train_generator, num_epochs=50)

Epoch:  1


KeyboardInterrupt: 

In [ ]:
# Testiranje i ocena modela.
average_loss = test_and_evaluate(test_generator)
print(f"Average L1 loss: {average_loss}")

In [ ]:
def generate_examples(model, test_generator, num_examples=5):
    count = 0
    for batch in test_generator:
        if count >= num_examples:
            break
        grayscale_images, rgb_images = batch
        predictions = model.predict(grayscale_images)
        for i in range(len(grayscale_images)):
            input_image = grayscale_images[i]
            target_image = rgb_images[i]
            predicted_image = predictions[i]
            display_list = [input_image, target_image, predicted_image]
            title = ["Input (Grayscale)", "Ground Truth (Color)", "Predicted (Color)"]
            plt.figure(figsize=(15, 5))
            for j in range(3):
                plt.subplot(1, 3, j + 1)
                plt.title(title[j])
                if j == 0:  # Grayscale image
                    plt.imshow(display_list[j], cmap='gray')
                else:
                    plt.imshow(display_list[j])
                plt.axis("off")
            plt.show()
            count += 1
            if count >= num_examples:
                break

In [ ]:
# Prikaz rezultata.
generate_examples(generator, test_generator, num_examples=5)